In [1]:
# import necessary libraries
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px

In [2]:
# read in the Data
data = pd.read_excel('/Users/tripatpanesar/Desktop/SARC 5400/SARC data.xlsx')

In [3]:
# make sure its read in properly
data.head(3)

,Country,Make,Model + Trim,Type,Price ($),Acceleration (s)
0,Japanese,Subaru,WRX Limited,Sedan,42000,5.4
1,German,Audi,SQ5 Sportback,SUV,70000,5.1
2,English,Range Rover,Velar,SUV,76000,5.2


In [4]:
# create Label for each data point
data['Label'] = data['Make'] + ' ' + data['Model + Trim']

In [5]:
# initialize Dash
app = dash.Dash(__name__)

# set layout, scatter plot first and then animation
app.layout = html.Div([
    dcc.Graph(id='scatter-plot'),
    dcc.Graph(id='animation-plot')
])

# app callback
@app.callback(
    Output('scatter-plot', 'figure'),
    [Input('scatter-plot', 'figure')]
)



# function to to update scatterplot
def update_scatter_plot(input_value):
    # initialize figure with proper attributes
    fig = px.scatter(
        data,
        text = 'Label',
        x='Price ($)',
        y='Acceleration (s)',
        color='Country', 
        hover_name='Label',
        hover_data=['Country', 'Price ($)','Type'],
        color_discrete_sequence=px.colors.qualitative.Plotly
    )

    # removing labels from points (it was too cluttery)
    fig.update_traces(
        textfont=dict(
            color='rgba(255, 255, 255, 0)' #setting color to 0 with 0 opacity
        ),
        marker=dict(
            size=10,
            line=dict(width=2, color='DarkSlateGrey') # tried to use markers before
        )
    )

    # setting size of scatterplot
    fig.update_layout(
        width=1300,
        height=900,
        title='Acceleration and Price of Cars by Country of Origin',
        yaxis=dict(range=[0, 8], dtick=1)
    )

    return fig

# callback to animation plot and connecting to scatterplot points
@app.callback(
    Output('animation-plot', 'figure'),
    [Input('scatter-plot', 'clickData')]
)



# function to update animation plot
def update_animation(clickData):
    if clickData:
        car_label = clickData['points'][0]['text']
        car_data = data[data['Label'] == car_label].iloc[0]
        acceleration_time = car_data['Acceleration (s)']
        
        car_image_url = 'https://png.pngtree.com/png-vector/20220625/ourmid/pngtree-car-transportation-green-cartoon-png-image_5320971.png'

        # initializing starting layout
        fig = go.Figure(
            data=[go.Scatter(
                x=[0], y=[0],
                mode='markers',
                marker=dict(size=40, opacity=0)
            )],
            # setting layout of added car
            layout=go.Layout(
                xaxis=dict(range=[0, 65], autorange=False),
                yaxis=dict(range=[-10, 5], autorange=False, showticklabels=False),
                title=f"{car_label}'s Acceleration: 0 to 60 mph in {acceleration_time} seconds",
                images=[{
                    'source': car_image_url,
                    'xref': 'x',
                    'yref': 'y',
                    'x': 0,
                    'y': 0,
                    'sizex': 5,
                    'sizey': 5,
                    'sizing': 'stretch',
                    'opacity': 1,
                    'layer': 'above'
                }],
                
                # adding blue and black shades to represent a road
                shapes=[
                    {
                        'type': 'rect',
                        'xref': 'paper',
                        'yref': 'y',
                        'x0': 0,
                        'y0': -10,
                        'x1': 1,
                        'y1': -5,
                        'fillcolor': 'black',
                        'opacity': 0.7,
                        'layer': 'below',
                        'line_width': 0,
                    },
                    {
                        'type': 'rect',
                        'xref': 'paper',
                        'yref': 'y',
                        'x0': 0,
                        'y0': -5,
                        'x1': 1,
                        'y1': 5,
                        'fillcolor': 'lightblue',
                        'opacity': 0.5,
                        'layer': 'below',
                        'line_width': 0,
                    }
                ]
            )
        )

        # initializing frames of animation
        frames = []
        for i in range(21):
            x_pos = 60 * i / 20
            frames.append(go.Frame(
                data=[go.Scatter(
                    x=[x_pos], y=[0],
                    mode='markers',
                    marker=dict(size=40, opacity=0)
                )],
                layout=go.Layout(
                    images=[{
                        'source': car_image_url,
                        'xref': 'x',
                        'yref': 'y',
                        'x': x_pos,
                        'y': 0,
                        'sizex': 5,
                        'sizey': 5,
                        'sizing': 'stretch',
                        'opacity': 1.0,
                        'layer': 'above'
                    }]
                )
            ))

        # setting frames to figure frames for proper animation
        fig.frames = frames

        # initialization and configuration of animation after pressing play
        fig.update_layout(
            updatemenus=[{
                'buttons': [{
                    'args': [None, {'frame': {'duration': acceleration_time * 1000 / 22, 'redraw': False},
                                    'fromcurrent': True, 'transition': {'duration': 500, 'easing': 'linear'}}],
                    'label': 'Play',
                    'method': 'animate'
                }],
                'direction': 'left',
                'pad': {'r': 10, 't': 87},
                'showactive': False,
                'type': 'buttons',
                'x': 0.1,
                'xanchor': 'left',
                'y': 0,
                'yanchor': 'top'
            }]
        )

        return fig
    
    return go.Figure()

In [6]:
# runnning Dash app and setting port to a localhost (changes)
if __name__ == '__main__':
    app.run_server(debug=True, host='127.0.0.1', port=8066)